# Trigger-Gaze $\Delta t$
The difference between trigger times and their closest gaze sample, to determine the value of hyperparameter `cnfg.MAX_GAZE_TO_TRIGGER_TIME_DIFF`, which determines if a gaze sample is close enough to a trigger to be considered "co-occurring".

In [ ]:
import numpy as np
import pandas as pd
import plotly.io as pio

import config as cnfg

pio.renderers.default = "notebook"      # or "browser"

### Parse Subjects

In [ ]:
from pipeline.parse_raw_data import parse_all_subjects

subjects = parse_all_subjects(cnfg.RAW_DATA_PATH, verbose=True)

### Calculate $\Delta t$
We calculate the temporal difference between each `gaze` measurement and `action` performed by the subject, and match `gaze` sample to `action` based on the minimum temporal difference.
We repeat this process twice: once allowing `NaN` values in the `gaze` data, and once excluding any `gaze` samples with `NaN` values.

In [ ]:
action_to_gaze_diffs = dict()
for subj in subjects:
    for tr in subj.get_trials():
        gaze = tr.get_gaze().sort_values("time")
        action = tr.get_actions().sort_values("time")
        for allow_nan_gaze in [True, False]:
            gaze_copy = gaze.copy()
            if not allow_nan_gaze:
                is_missing_left = gaze[['left_x', 'left_y']].isna().any(axis=1)
                is_missing_right = gaze[['right_x', 'right_y']].isna().any(axis=1)
                is_missing = is_missing_left & is_missing_right
                gaze_copy = gaze_copy.loc[~is_missing]
            gaze_times = gaze_copy['time'].values
            action_times = action['time'].values
            diffs = np.abs(gaze_times[:, None] - action_times[None, :])  # shape (gaze, actions)
            min_diff = diffs.min(axis=0)  # minimum difference for each action time; shape (actions,)
            action_to_gaze_diffs[(subj.id, tr.trial_num, allow_nan_gaze)] = min_diff


action_to_gaze_diffs = (
    pd.Series(action_to_gaze_diffs)
    .explode()
    .dropna()
    .reset_index(drop=False)
    .rename(columns={"level_0": "subject", "level_1": "trial", "level_2": "allow_nan_gaze", 0: "time_diff"})
)

action_to_gaze_diffs

### Analyze $\Delta t$
We check the minimum and maximum `$Delta t$` values

In [ ]:
action_to_gaze_diffs.groupby(["subject", "trial", "allow_nan_gaze"]).agg(
    min_time_diff=("time_diff", "min"),
    max_time_diff=("time_diff", "max"),
).reset_index()

In [ ]:
action_to_gaze_diffs.groupby("allow_nan_gaze")["time_diff"].value_counts()